### 3.1. Show 10 dòng đầu của file:
- 20251127_camera.xlsx
- 20251127_system.xlsx
- mapping.xlsx

In [2]:
import pandas as pd

# =========================
# CONFIG
# =========================
files = {
    "CAMERA": "3/3_1/input_data/20251127_camera.xlsx",
    "SYSTEM": "3/3_1/input_data/20251127_system.xlsx",
    "MAPPING": "3/3_1/input_data/mapping.xlsx"
}

# =========================
# READ & SHOW FIRST 10 ROWS
# =========================
for name, path in files.items():
    print(f"\n===== {name} — 10 dòng đầu tiên =====")
    try:
        df = pd.read_excel(path)
        print(df.head(10))
    except Exception as e:
        print(f"Lỗi đọc file {path}: {e}")


===== CAMERA — 10 dòng đầu tiên =====
   Mã đường      Tên đường   Năm  Xe máy  Xe ô tô  Xe buýt  Xe tải nặng  \
0       101  Phố Báo Khánh  2024     154      144       37           21   
1       101  Phố Báo Khánh  2024     180      116        8           20   
2       101  Phố Báo Khánh  2024     224      110        9            3   
3       101  Phố Báo Khánh  2024      38       83       34           18   
4       101  Phố Báo Khánh  2024     202      115       19           28   
5       101  Phố Báo Khánh  2024      85      104       47           19   
6       101  Phố Báo Khánh  2024     361      337       89           38   
7       101  Phố Báo Khánh  2024     720       97      195          139   
8       101  Phố Báo Khánh  2024    1132      641      145           37   
9       101  Phố Báo Khánh  2024    1129      590      119           46   

   Xe tải nhẹ        Loại ngày    Giờ        Ngày  
0          35  Ngày trong tuần  00:00  27/11/2025  
1          22  Ngày trong tuần 

### 3.2. Ánh xạ lưu lượng phương tiện từ đường camera giao thông thực tế => đoạn đường ở trong CSDL.
- Case 1: Đường Camera có đủ dữ liệu của khung 24h.

In [9]:
import os
import pandas as pd

# ==========================================================
# 1) CONFIG
# ==========================================================
input_dir = "3/3_1/input_data"
output_dir = "3/3_1/output_data"
os.makedirs(output_dir, exist_ok=True)

camera_file  = os.path.join(input_dir, "20251127_camera.xlsx")
system_file  = os.path.join(input_dir, "20251127_system.xlsx")
mapping_file = os.path.join(input_dir, "mapping.xlsx")

output_file = os.path.join(output_dir, "20251127_system_mapped_case_1.xlsx")


# ==========================================================
# 2) LOAD DATA
# ==========================================================
print("=== Đang đọc file camera... ===")
df_camera = pd.read_excel(camera_file)

print("=== Đang đọc file system... ===")
df_system = pd.read_excel(system_file)

print("=== Đang đọc file mapping... ===")
df_mapping = pd.read_excel(mapping_file)

print(">>> Đọc file hoàn tất\n")


# ==========================================================
# 3) TIỀN XỬ LÝ
# ==========================================================
# Chuẩn hóa kiểu dữ liệu
df_camera["Mã đường"] = df_camera["Mã đường"].astype(str)
df_system["Mã đường"] = df_system["Mã đường"].astype(str)
df_mapping["Mã đường"] = df_mapping["Mã đường"].astype(str)
df_mapping["Mã đoạn đường"] = df_mapping["Mã đoạn đường"].astype(str)

# Tạo dict ánh xạ: road_code → list(segment_code)
map_dict = (
    df_mapping.groupby("Mã đường")["Mã đoạn đường"]
    .apply(list)
    .to_dict()
)

print("=== Sinh bảng ánh xạ hoàn tất ===\n")


# ==========================================================
# 4) CHUẨN HÓA CAMERA THEO GIỜ
# ==========================================================
# Đảm bảo group đúng theo từng Mã đường + Giờ
camera_grouped = df_camera.groupby(["Mã đường", "Giờ"], as_index=False).agg({
    "Xe máy": "sum",
    "Xe ô tô": "sum",
    "Xe buýt": "sum",
    "Xe tải nặng": "sum",
    "Xe tải nhẹ": "sum"
})

print("=== Gom nhóm camera theo Mã đường + Giờ hoàn tất ===\n")


# ==========================================================
# 5) MAP DỮ LIỆU CAMERA → SYSTEM
# ==========================================================
print("=== Bắt đầu ánh xạ dữ liệu camera vào system ===")

records_updated = 0

for idx, row in df_system.iterrows():
    segment_code = row["Mã đường"]          # ví dụ: 1000561
    hour = row["Giờ"]                       # ví dụ: 07:00

    # Tìm mã đường gốc camera
    found = None
    for road, segment_list in map_dict.items():
        if segment_code in segment_list:
            found = road
            break

    if not found:
        # Không ánh xạ → bỏ qua
        continue

    # Lấy dữ liệu camera tương ứng Mã đường + Giờ
    matched = camera_grouped[
        (camera_grouped["Mã đường"] == found) &
        (camera_grouped["Giờ"] == hour)
    ]

    if matched.empty:
        continue

    camera_row = matched.iloc[0]

    # Ghi dữ liệu vào system
    df_system.at[idx, "Xe máy"] = camera_row["Xe máy"]
    df_system.at[idx, "Xe ô tô"] = camera_row["Xe ô tô"]
    df_system.at[idx, "Xe buýt"] = camera_row["Xe buýt"]
    df_system.at[idx, "Xe tải nặng"] = camera_row["Xe tải nặng"]
    df_system.at[idx, "Xe tải nhẹ"] = camera_row["Xe tải nhẹ"]

    records_updated += 1

print(f"=== Hoàn tất ánh xạ. Tổng số record được cập nhật: {records_updated} ===\n")


# ==========================================================
# 6) EXPORT FILE
# ==========================================================
df_system.to_excel(output_file, index=False)
print(f"=== ĐÃ XUẤT FILE HOÀN TẤT: {output_file} ===")

=== Đang đọc file camera... ===
=== Đang đọc file system... ===
=== Đang đọc file mapping... ===
>>> Đọc file hoàn tất

=== Sinh bảng ánh xạ hoàn tất ===

=== Gom nhóm camera theo Mã đường + Giờ hoàn tất ===

=== Bắt đầu ánh xạ dữ liệu camera vào system ===
=== Hoàn tất ánh xạ. Tổng số record được cập nhật: 7656 ===

=== ĐÃ XUẤT FILE HOÀN TẤT: 3/3_1/output_data/20251127_system_mapped_case_1.xlsx ===


#### 3.2. In 10 dòng đâu tiên của file OUTPUT

In [10]:
import pandas as pd

# =========================
# CONFIG
# =========================
file_path = "3/3_1/output_data/20251127_system_mapped_case_1.xlsx"

# =========================
# READ & SHOW FIRST 10 ROWS
# =========================
df = pd.read_excel(file_path)

print("\n===== 10 dòng đầu tiên =====\n")
print(df.head(10))


===== 10 dòng đầu tiên =====

   Mã đường       Tên đường   Năm  Xe máy  Xe ô tô  Xe buýt  Xe tải nặng  \
0   1000001  Phố Tràng Tiền  2024     240       58       29           16   
1   1000001  Phố Tràng Tiền  2024     248       60        2           19   
2   1000001  Phố Tràng Tiền  2024     138       46       31           12   
3   1000001  Phố Tràng Tiền  2024     254       67       32           28   
4   1000001  Phố Tràng Tiền  2024     150       34       30           22   
5   1000001  Phố Tràng Tiền  2024     119       45       34           40   
6   1000001  Phố Tràng Tiền  2024     229      161        9           30   
7   1000001  Phố Tràng Tiền  2024     865      362      130          165   
8   1000001  Phố Tràng Tiền  2024    1048      224      202           54   
9   1000001  Phố Tràng Tiền  2024     825      878       52          139   

   Xe tải nhẹ        Loại ngày    Giờ  
0           7  Ngày trong tuần  00:00  
1          19  Ngày trong tuần  01:00  
2          2